## Mental Health Discussion Analyzer Web Scraper
#### Author
**Name:** Andres Figueroa  
**Email:** andresfigueroa@brandeis.edu

#### Project Description
The purpose of this file is to collects data by web scraping. I am doing this on .ipynb file because this is my first time web scraping. I like the markdown, makes notes look nice.

---

#### Importing Libraries

In [ ]:
import praw
import pandas as pd

The `praw` library is the Python Reddit API Wrapper (PRAW). PRAW is the Reddit API, this will allow me acces the data I want from Reddit.

**Note:** I can use the `requests` and `BeautifulSoup` for raw HTML scraping, but I guess the API is easier to use.

---

### Understanding the Target Data (What We Are Looking For)

I was thinking about scraping data from the following Reddit communities:
- r/mentalhealth
- r/depression
- r/anxiety

**Note:** After viewing the number of followers and reading some of the posts, I have decided to just scrape data from just `r/mentalhealth` as posts covers a variety of disorders and situations.

As for the data that I will be collecting, I will will collect:
- Post Body Text
- Timestamp
- Number of Upvotes
- Number of Comments

**Note:** From this data I am hoping to better understand what people are talking about, where conversations or posts are positive, negative, or neutral.